在Python中执行整数和浮点数的数学运算时很简单的。  
尽管如此，如果你需要
# 执行分数、数组或者是日期和时间的运算的话，就得做更多的工作了。本章集中讨论的就是这些主题。

## 3.1 数字的四舍五入

想对浮点数执行精度的舍入运算,对于简单的舍入运算使用内置的`round`函数即可

In [2]:
print(round(1.23, 1)) #保留1位
round(1.23561, 3)

1.2


1.236

当一个值刚好在两个边界的中间的时候， round 函数返回离它最近的偶数。也就 是说，对 1.5 或者 2.5 的舍入运算都会得到 2。
传给 round() 函数的 ndigits 参数可以是负数，这种情况下，舍入运算会作用在 十位、百位、千位等上面。比如
```python
a = 1627730
round(a, -2) #1627770 ， 作用在百位上
```

**不要将舍入和格式化输出搞混淆**了。如果你的目的只是简单的输出一定宽度的数， 你不需要使用 round() 函数。  
而仅仅只需要在格式化的时候指定精度即可。比如

In [9]:
x = 1.23456
print(format(x, '0.2f')) #保留2位
'value is {:0.3f}'.format(x)  #冒号后面的.f表示将该参数格式化为浮点数，并保留f位小数

1.23


'value is 1.235'

同样，不要试着去舍入浮点值来” 修正” 表面上看起来正确的问题。比如，你可能 倾向于这样做：

In [12]:
a = 2.1 
b = 4.2
c = a+b
print(c)
c = round(c, 2) #企图修复
c

6.300000000000001


6.3

对于大多数使用到浮点的程序，没有必要也不推荐这样做。  
尽管在计算的时候会有 一点点小的误差，但是这些小的误差是能被理解与容忍的。如果不能允许这样的小误 差 (比如涉及到金融领域)，  
那么就得考虑使用 decimal 模块了，下一节我们会详细讨 论。

## 3.2执行精确的浮点数运算

需要对浮点数执行精确的计算操作，并且不希望有任何小误差的出现  
浮点数的一个普遍问题是它们并不能精确的表示十进制数。并且，即使是最简单的 数学运算也会产生小的误差，比如上述`a+b == 6.3 会返回False`

这些错误是由底层 CPU 和 IEEE 754 标准通过自己的浮点单位去执行算术时的特征。  
由于 Python 的浮点数据类型使用底层表示存储数据，因此你没办法去避免这样的误差。  
如果你想更加精确 (并能容忍一定的性能损耗)，你可以使用 decimal 模块：

In [14]:
from decimal import Decimal
a = Decimal('2.1')
b = Decimal('4.2')
a+b

Decimal('6.3')

初看起来， 上面的代码好像有点奇怪，比如我们用字符串来表示数字。然而 Decimal 对象会像普通浮点数一样的工作 (支持所有的常用数学运算)。  
如果你打印它们或者在字符串格式化函数中使用它们，看起来跟普通数字没什么两样。  
decimal模块的一个主要特征是允许你控制计算的每一方面，包括数字位数和四舍五入运算。为了这样做，你先得创建一个本地上下文并更改它的设置，比如：

In [15]:
a = Decimal('1.3')
b = Decimal('1.7')
print( a / b)

0.7647058823529411764705882353


In [17]:
from decimal import localcontext
with localcontext() as ctx: 
    ctx.prec = 3 # 四舍五入 保留3位
    print(a / b)

0.765


decimal 模块实现了 IBM 的” 通用小数运算规范”。不用说，有很多的配置选项这 本书没有提到。

Python 新手会倾向于使用 decimal 模块来处理浮点数的精确运算。然而，先理解 你的应用程序目的是非常重要的。如果你是在做科学计算或工程领域的计算、电脑绘 图，或者是科学领域的大多数运算，那么使用普通的浮点类型是比较普遍的做法。其 中一个原因是， **在真实世界中很少会要求精确到普通浮点数能提供的 17 位精度**。 因 此，计算过程中的那么一点点的误差是被允许的。第二点就是，原生的浮点数计算要 快的多 有时候你在执行大量运算的时候速度也是非常重要的。

即便如此，你却不能完全忽略误差。数学家花了大量时间去研究各类算法，有些处 理误差会比其他方法更好。你也得注意下减法删除 已经大数和小数的加分运算所带来 的影响。比如：

In [18]:
nums = [1.23e+18, 1, -1.23e+18]
sum(nums) # Notice how 1 disappears  ？？

0.0

上面的错误可以利用 math.fsum() 所提供的更精确计算能力来解决：

In [19]:
import math
math.fsum(nums)

1.0

然而，对于其他的算法，你应该仔细研究它并理解它的误差产生来源。  
总的来说， decimal 模块主要用在涉及到金融的领域。在这类程序中，哪怕是一点小小的误差在计算过程中蔓延都是不允许的。  
因此， decimal 模块为解决这类问题提供了方法。当 Python 和数据库打交道的时候也通常会遇到 Decimal 对象，并且，通常也是在处理金融数据的时候

## 3.3数字的格式化输出
问题：你需要将数字格式化后输出， 并控制数字的位数、对齐、千位分隔符和其他的细节。
格式化输出单个数字的时候，可以使用内置的`format`

x = 1234.56789
```python
# Two decimal places of accuracy
format(x, '0.2f')  #'1234.57'

# Right justified in 10 chars, one-digit accuracy

format(x, '>10.1f') #' 1234.6'

# Left justified

format(x, '<10.1f') #'1234.6 '

 # Centered

format(x, '^10.1f') #' 1234.6 '

 # Inclusion of thousands separator 包含千位分隔符

format(x, ',') #'1,234.56789'

format(x, '0,.1f') #'1,234.6'
```

如果你想使用指数记法，将 f 改成 e 或者 E(取决于指数输出的大小写形式)。比如：

format(x, 'e')    #'1.234568e+03'  
format(x, '0.2E')  #'1.23E+03'  
同时指定宽度和精度的一般形式是 '[<>ˆ]?width[,]?(.digits)?' ，其中 width 和 digits 为整数，  
？代表可选部分。同样的格式也被用在字符串的 format() 方法中。 比如：  
`The value is {:0,.2f}'.format(x)`   # 'The value is 1,234.57'

## 3.4 二 八 十六进制整数
问题：你需要转换或者输出使用二进制，八进制或十六进制表示的整数。  
解决方案：为了将整数转换为二进制、八进制或十六进制的文本串，可以分别使用 `bin()` , `oct()` 或 `hex()` 函数：

In [20]:
x = 17
print(bin(x))  #不想要0b之类的前缀 可以使用format函数，如 format(x, 'b')
print(oct(x))
print(hex(x))

0b10001
0o21
0x11


整数是有符号的，所以如果你在处理负数的话，输出结果会包含一个负号。  
如果你想产生一个无符号值，你需要增加一个指示最大位长度的值。比如为了显示 32 位的值，可以像下面这样写：

In [21]:
x = -1234
format(2**32 + x, 'b')

'11111111111111111111101100101110'

为了以不同的**进制 转换 整数字符串**，简单的使用带有进制的 int() 函数即可：

In [24]:
int('10011010010', 2)

1234

大多数情况下处理二进制、八进制和十六进制整数是很简单的。只要记住这些转换 属于整数和其对应的文本表示之间的转换即可。永远只有一种整数类型  
最后，使用八进制的程序员有一点需要注意下。Python 指定八进制数的语法跟其 他语言稍有不同, `os.chmod('xxx.sh', 0o755)`  八进制的前缀是0o

## 3.5字节到大整数的打包与解包  
问题:你有一个字节字符串并想将它解压成一个整数。或者，你需要将一个大整数转换为 一个字节字符串。  
假设你的程序需要处理一个拥有 128 位长的 16 个元素的字节字符串。比如：

In [29]:
xdata = b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004' # \x 值为16位16进制的字符
print(len(xdata))

16


为了将 bytes 解析为整数，使用 int.from bytes() 方法，并像下面这样指定字节 顺序

In [30]:
print(int.from_bytes(xdata, 'little'))
print(int.from_bytes(xdata, 'big'))

69120565665751139577663547927094891008
94522842520747284487117727783387188


为了将一个大整数转换为一个字节字符串，使用 int.to bytes() 方法，并像下面 这样指定字节数和字节顺序：


In [31]:
x = 94522842520747284487117727783387188
x.to_bytes(16, 'big')

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

大整数和字节字符串之间的转换操作并不常见。然而，在一些应用领域有时候也会 出现，比如密码学或者`网络`。例如，`IPv6 网络地址使用一个 128 位的整数表示。如果 你要从一个数据记录中提取这样的值的时候`，你就会面对这样的问题。  
作为一种替代方案，你可能想使用 6.11 小节中所介绍的 struct 模块来解压字节。 这样也行得通，不过利用 struct 模块来解压对于整数的大小是有限制的。因此，你可 能想解压多个字节串并将结果合并为最终的结果，就像下面这样：

In [32]:
import struct
hi, lo = struct.unpack('>QQ', xdata)
(hi << 64) + lo

94522842520747284487117727783387188

字节顺序规则 (little 或 big) 仅仅指定了构建整数时的字节的低位高位排列方式。  
 我们从下面精心构造的 16 进制数的表示中可以很容易的看出来：

In [34]:
x = 0x01020304
print(x.to_bytes(4, 'big'))
print(x.to_bytes(4, 'little'))

b'\x01\x02\x03\x04'
b'\x04\x03\x02\x01'


如果你试着将一个整数打包为字节字符串， 那么它就不合适了， 你会得到一个错 误。  
如果需要的话，你可以使用 int.bit length() 方法来决定需要多少字节位来存储 这个值。

## 3.6 复数的数学运算
问题：你写的最新的网络认证方案代码遇到了一个难题，并且你唯一的解决办法就是使用复数空间。  
再或者是你仅仅需要使用复数来执行一些计算操作。  
复数可以用使用函数 complex(real, imag)或者是带有后缀j的浮点数来指定。 

In [36]:
a = complex(2, 4)
b = 3 - 5j
print(a, b)

(2+4j) (3-5j)


对应的实部、虚部和共轭复数可以很容易的获取。就像下面这样：

In [38]:
print(a.real, a.imag)
a.conjugate()

2.0 4.0


(2-4j)